In [1]:
import sys 
import os
from bh24_literature_mining.europepmc_api import EuropePMCClient
from bh24_literature_mining.utils import load_biotools_pub
from bh24_literature_mining.utils import load_biotools_from_zip
from bh24_literature_mining.utils import load_biotools_from_json
import numpy as np
import json
import pandas as pd
import requests
import igraph as ig


In [ ]:
_PY4CYTOSCAPE = 'git+https://github.com/cytoscape/py4cytoscape@1.7.0' # optional
exec(requests.get("https://raw.githubusercontent.com/cytoscape/jupyter-bridge/master/client/p4c_init.py").text)
IPython.display.Javascript(_PY4CYTOSCAPE_BROWSER_CLIENT_JS) # Start browser client

ERROR: Can not perform a '--user' install. User site-packages are not visible in this virtualenv.


SystemExit: 1

/home/ucloud/.cache/pypoetry/virtualenvs/bh24-literature-mining-9a6zG5on-py3.12/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3040: UserWarning: Unknown failure executing module: <pip>
  warn('Unknown failure executing module: <%s>' % mod_name)


ModuleNotFoundError: No module named 'py4cytoscape'

In [ ]:
import py4cytoscape as p4c
p4c.cytoscape_ping()
p4c.cytoscape_version_info()

You are connected to Cytoscape!


{'apiVersion': 'v1',
 'cytoscapeVersion': '3.7.2',
 'automationAPIVersion': '1.11.0',
 'py4cytoscapeVersion': '1.11.0'}

In [2]:
from bh24_literature_mining.europepmc_api import Article

tools = Article.read_cites_from_json("../var/biotools_cites.json")


print(f'Loaded {len(tools)} tools.')

Loaded 9453 tools.


### Remove preprints from articles in every tool

For published articles there can also be preprints included since DOI and other IDs are different from the final publication to the preprint.

In [3]:
for tool in tools:
    tool["articles"] = [
        article for article in tool["articles"] if article.pubType != "preprint"
    ]

print(f"Filtered out preprints. Now {len(tools)} tools.")

Filtered out preprints. Now 9453 tools.


### Get all publication IDs

In [4]:
all_ids = []
for tool in tools:
    for article in tool['articles']:
        all_ids.append(article.id)

all_ids = set(all_ids)

print(f"Total number of articles: {len(all_ids)}")

Total number of articles: 366828


### Create binary matrix for tools vs. articles

In [7]:
tool_names = [tool['name'] for tool in tools]
matrix = pd.DataFrame(0, index=tool_names, columns=sorted(set(all_ids)))
i = 0
for tool in tools:
    print(f'{i}/{len(tools)}')
    i = i + 1
    for article in tool['articles']:   
        matrix.loc[ tool['name'], article.id] = 1

0/500
1/500
2/500
3/500
4/500
5/500
6/500
7/500
8/500
9/500
10/500
11/500
12/500
13/500
14/500
15/500
16/500
17/500
18/500
19/500
20/500
21/500
22/500
23/500
24/500
25/500
26/500
27/500
28/500
29/500
30/500
31/500
32/500
33/500
34/500
35/500
36/500
37/500
38/500
39/500
40/500
41/500
42/500
43/500
44/500
45/500
46/500
47/500
48/500
49/500
50/500
51/500
52/500
53/500
54/500
55/500
56/500
57/500
58/500
59/500
60/500
61/500
62/500
63/500
64/500
65/500
66/500
67/500
68/500
69/500
70/500
71/500
72/500
73/500
74/500
75/500
76/500
77/500
78/500
79/500
80/500
81/500
82/500
83/500
84/500
85/500
86/500
87/500
88/500
89/500
90/500
91/500
92/500
93/500
94/500
95/500
96/500
97/500
98/500
99/500
100/500
101/500
102/500
103/500
104/500
105/500
106/500
107/500
108/500
109/500
110/500
111/500
112/500
113/500
114/500
115/500
116/500
117/500
118/500
119/500
120/500
121/500
122/500
123/500
124/500
125/500
126/500
127/500
128/500
129/500
130/500
131/500
132/500
133/500
134/500
135/500
136/500
137/500
138/50

In [8]:
matrix

,10811817,11125070,11178258,11574056,11597340,11869452,12019020,12171605,12877744,14611664,...,PPR7024,PPR7025,PPR7026,PPR7027,PPR7028,PPR7029,PPR7032,PPR7033,PPR7034,PPR7035
Seurat,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Bakta,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
gget,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SIBiLS,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Variomes,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HGMD,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
HGMD,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
mCSM-membrane,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GPCRsignal,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Multiple matrix operations
Filtering for duplicates and filtering

In [9]:
matrix.drop_duplicates(inplace=True)
matrix_filt = matrix.loc[:, matrix.sum(axis=0) > 1]
adj = np.dot(matrix.values, matrix.values.T)
adj_df = pd.DataFrame(adj, index=matrix.index, columns=matrix.index)
np.fill_diagonal(adj_df.values, 0)

In [45]:
# Get the values as np.array, it's more convenient.
A = adj_df.values

# Create graph, A.astype(bool).tolist() or (A / A).tolist() can also be used.
g = ig.Graph.Adjacency((A > 0).tolist())

# Add edge weights and node labels.
g.es['weight'] = A[A.nonzero()]
g.vs['name'] = adj_df.columns  # or a.index/a.columns

In [47]:
# Check Cytoscape connection
p4c.cytoscape_ping()

# Convert igraph graph to Cytoscape and create the network
p4c.create_network_from_igraph(g, title="Comention Network", collection="Comention Network Collection")

# Set node and network visual properties
#p4c.set_node_color_bypass(pmids, "red")
p4c.set_node_shape_default("ELLIPSE")
p4c.set_node_width_default(30)
p4c.set_node_height_default(30)

# Compute shortest paths using igraph and display in console or store as needed


You are connected to Cytoscape!
Applying default style...
Applying preferred layout
style_name not specified, so updating "default" style.
style_name not specified, so updating "default" style.
style_name not specified, so updating "default" style.
style_name not specified, so updating "default" style.
style_name not specified, so updating "default" style.


''

In [5]:
tools = tools[:500]